# 최종

# business.json
- PA 지역 Philadelphia 도시
- 필요없는 column 제거
- 음식점 카테고리만 선택

In [136]:
import pandas as pd
import json

# JSON Lines 형식으로 business.json 불러오기
df_B = pd.read_json("yelp_academic_dataset_business.json", lines=True)

In [147]:
business_df=df_B.copy()

In [148]:
# 제거할 컬럼 목록
drop_cols = ['postal_code', 'latitude', 'longitude', 'attributes', 'hours']

# 컬럼 제거
business_df = business_df.drop(columns=drop_cols)

# "city" 컬럼 내 값을 통일: Philadelphia 관련은 모두 "Philadelphia"로 변경
business_df.loc[business_df['city'].str.lower().str.contains("philadelphia", na=False),'city'] = "Philadelphia"

# 카테고리 불러오기 함수
def load_categories(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        return set(line.strip() for line in f if line.strip())

# 파일 경로
food_path = "food.txt"
restaurant_path = "restaurant.txt"

# 타겟 카테고리 준비
food_categories = load_categories(food_path)
restaurant_categories = load_categories(restaurant_path)
target_categories = food_categories.union(restaurant_categories)

# 필터링 함수
def category_match(row):
    if isinstance(row, str):
        biz_categories = set(cat.strip() for cat in row.split(","))
        return bool(biz_categories & target_categories)
    return False

# ✅ 필터링 적용
business_food_df = business_df[business_df["categories"].apply(category_match)]

# 가장 많은 state 값 구하기 - PA
top_state = business_food_df['state'].value_counts().idxmax()
business_pa_df = business_food_df[business_food_df['state'] == top_state]
business_paph_df = business_pa_df[business_pa_df['city'] == "Philadelphia"]

In [167]:
output_path = "business.json"

# ✅ JSON Lines 형식으로 저장
business_paph_df.to_json(output_path, orient="records", lines=True, force_ascii=False)

print(f"✅ 총 {len(business_paph_df)}개 항목이 '{output_path}'에 저장되었습니다.")

✅ 총 6911개 항목이 'business.json'에 저장되었습니다.


# review.json
- PA 지역 philadelphia 도시
- 필요없는 column 제거
- 선호도 평점 -> 감성분석 (1,2 부정 | 3 중립 | 4,5 긍정)

In [141]:
import pandas as pd

# 파일 경로
file_path = "yelp_academic_dataset_review.json"

# chunk 단위로 전체 데이터 읽고 합치기
chunk_size = 100000
chunks = pd.read_json(file_path, lines=True, chunksize=chunk_size)

# 모든 chunk를 리스트에 담아서 연결
df_r = pd.concat(chunk for chunk in chunks)

In [159]:
df_review = df_r.copy()

In [160]:
import pandas as pd
import json

# PA - Philadelphia 리뷰
business_ids = set(business_paph_df["business_id"])

# business_id 기준 필터링
df_review = df_review[df_review["business_id"].isin(business_ids)]

# funny와 cool 컬럼 제거
df_review = df_review.drop(columns=["funny", "cool"])

In [162]:
# 감성 분류 함수 정의
def classify_sentiment(stars):
    if stars in [1, 2]:
        return 'Negative'
    elif stars == 3:
        return 'Neutral'
    elif stars in [4, 5]:
        return 'Positive'
    else:
        return 'Unknown'
    
df_review['sentiment'] = df_review['stars'].apply(classify_sentiment)

In [168]:
df_review.to_json("review.json", orient="records", lines=True, force_ascii=False)

# user.json
- 필요 없는 column 제거
- PA지역 Philadelphia 도시에 review 5개 이상 작성한 사용자

In [156]:
import pandas as pd

# 파일 경로 설정
file_path = "yelp_academic_dataset_user.json"

# 1. chunk 단위로 불러오기
chunks = pd.read_json(file_path, lines=True, chunksize=100000)

# 2. 모든 chunk 읽고 합치기
df_u = pd.concat(chunk for chunk in chunks)

In [157]:
df_user = df_u.copy()
df_user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1987897 entries, 0 to 1987896
Data columns (total 22 columns):
 #   Column              Dtype  
---  ------              -----  
 0   user_id             object 
 1   name                object 
 2   review_count        int64  
 3   yelping_since       object 
 4   useful              int64  
 5   funny               int64  
 6   cool                int64  
 7   elite               object 
 8   friends             object 
 9   fans                int64  
 10  average_stars       float64
 11  compliment_hot      int64  
 12  compliment_more     int64  
 13  compliment_profile  int64  
 14  compliment_cute     int64  
 15  compliment_list     int64  
 16  compliment_note     int64  
 17  compliment_plain    int64  
 18  compliment_cool     int64  
 19  compliment_funny    int64  
 20  compliment_writer   int64  
 21  compliment_photos   int64  
dtypes: float64(1), int64(16), object(5)
memory usage: 333.7+ MB


In [ ]:
# 제거할 컬럼 리스트
drop_columns = [
    "yelping_since", "funny", "cool", "elite", "friends", "fans", 
    "compliment_hot", "compliment_more", "compliment_profile", "compliment_cute",
    "compliment_list", "compliment_note", "compliment_plain", "compliment_cool",
    "compliment_funny", "compliment_writer", "compliment_photos"
]

# 제거 수행
df_user = df_user.drop(columns=drop_columns)

# user_id별 PA-Philadelphia 리뷰 수 세기
review_counts = df_review['user_id'].value_counts()

# 리뷰가 5개 이상인 user_id만 추출
user_ids_5plus = review_counts[review_counts >= 5].index

# df_user에서 해당 user_id에 해당하는 사용자 정보만 필터링
df_user = df_user[df_user['user_id'].isin(user_ids_5plus)]

In [165]:
df_user.info()

<class 'pandas.core.frame.DataFrame'>
Index: 28518 entries, 1 to 1908265
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   user_id        28518 non-null  object 
 1   name           28518 non-null  object 
 2   review_count   28518 non-null  int64  
 3   useful         28518 non-null  int64  
 4   average_stars  28518 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 1.3+ MB


In [166]:
df_user.to_json("user.json", orient="records", lines=True, force_ascii=False)